In [1]:
using SparseArrays
using LinearAlgebra
using NLopt
using PyPlot
using KrylovKit
using FiniteDifferences
using Revise
using TopologyOptimizationHelper

In [208]:
function Maxwell2d(Lx, Ly, ε, ω; dpml=2, resolution=20, Rpml=1e-20, ω_pml=ω)
    ε = vec(ε)
    # PML σ = σ₀ x²/dpml², with σ₀ chosen so that the round-trip reflection is Rpml
    σ₀ = -log(Rpml) / (4dpml^3/3)
    
    M = round(Int, (Lx+2dpml) * resolution)
    N = round(Int, (Ly+2dpml) * resolution)
    dx = (Lx+2dpml) / (M+1)
    dy = (Ly+2dpml) / (N+1)
    x = (1:M) * dx # x grid
    y = (1:N) * dy # y grid
    x′ = @. ((0:M) + 0.5) * dx # 1st-derivative grid points
    y′ = @. ((0:N) + 0.5) * dy
    
    # 1st-derivative matrices
    ox = ones(M) / dx
    oy = ones(N) / dy
    Dx = spdiagm(M+1,M, -1 => -ox, 0 => ox)
    Dy = spdiagm(N+1,N, -1 => -oy, 0 => oy)
    
    # PML complex "stretch" factors 1/(1+iσ/ω_pml) at both x and x' points:
    σx = [ξ < dpml ? σ₀*(dpml-ξ)^2 : ξ > Lx+dpml ? σ₀*(ξ-(Lx+dpml))^2 : 0.0 for ξ in x]
    sqrtΣx = spdiagm(@. sqrt(inv(1 + (im/ω_pml)*σx)))
    σx′ = [ξ < dpml ? σ₀*(dpml-ξ)^2 : ξ > Lx+dpml ? σ₀*(ξ-(Lx+dpml))^2 : 0.0 for ξ in x′]
    Σx′ = spdiagm(@. inv(1 + (im/ω_pml)*σx′))
    # similarly for y and y':
    σy = [ξ < dpml ? σ₀*(dpml-ξ)^2 : ξ > Ly+dpml ? σ₀*(ξ-(Ly+dpml))^2 : 0.0 for ξ in y]
    sqrtΣy = spdiagm(@. sqrt(inv(1 + (im/ω_pml)*σy)))
    σy′ = [ξ < dpml ? σ₀*(dpml-ξ)^2 : ξ > Ly+dpml ? σ₀*(ξ-(Ly+dpml))^2 : 0.0 for ξ in y′]
    Σy′ = spdiagm(@. inv(1 + (im/ω_pml)*σy′))
    
    # stretched 2nd-derivative matrices
    D2x = sqrtΣx * Dx' * Σx′ * Dx * sqrtΣx
    D2y = sqrtΣy * Dy' * Σy′ * Dy * sqrtΣy
    
    # combine x and y with Kronecker products
    Ix = spdiagm(ones(M))
    Iy = spdiagm(ones(N))
    x = x .- dpml
    y = y .- dpml
    return kron(Ix, D2y) + kron(D2x, Iy) .- ω^2 .* spdiagm(ε), x, y
end

Maxwell2d (generic function with 1 method)

In [222]:
# Our favorite parameters for testing
# Re-run to randomize
Lx = 20
Ly = 20
ε = rand(480, 480) .* 11 .+ 1
δε = randn(size(ε)) * 1e-5
ω = 2π
δω = randn() * 1e-5

A, x, y = Maxwell2d(Lx, Ly, ε, ω)
new_ε_A, _, _ = Maxwell2d(Lx, Ly, ε + δε, ω)
new_ω_A, _, _ = Maxwell2d(Lx, Ly, ε, ω + δω; ω_pml = ω)

M, N = length(x), length(y)
b = zeros(N, M)
b[N÷2,M÷2] = 1;

In [69]:
# Exact gradient of LDOS versus numerical gradient
LDOS, ∇LDOS = ∇_ε_LDOS(A, ω, vec(b))
new_ε_LDOS, _ = ∇_ε_LDOS(new_ε_A, ω, vec(b))

@show new_ε_LDOS - LDOS
∇LDOS' * vec(δε)

new_ε_LDOS - LDOS = -9.001514080344067e-8


-9.004113304597974e-8

In [70]:
# Exact gradient of eigenvalue versus numerical gradient
val, gradient = Eigengradient(A, vec(ε), ω, vec(b))
new_val, _ = Eigengradient(new_ε_A, vec(ε + δε), ω, vec(b))
@show real(new_val) - real(val)
dot(vec(δε), real(gradient))

real(new_val) - real(val) = 8.01746265022274e-6


8.014588210906811e-6

In [71]:
# Exact gradient of LDOS versus Richardson extrapolation
∇LDOS = ∇_ω_LDOS(A, vec(ε), ω, vec(b))
new_ω_LDOS, _ = ∇_ε_LDOS(new_ω_A, ω, vec(b))
@show new_ω_LDOS - LDOS
∇LDOS[1]' * real(δω)

new_ω_LDOS - LDOS = -7.494208767336134e-9


-7.520649854193518e-9

In [217]:
ω₀, ∂ω_∂ε = Eigengradient(A, vec(ε), ω, vec(b))
A₀, _, _ = Maxwell2d(Lx, Ly, vec(ε), real(ω₀); ω_pml=ω)
true_LDOS, ∂LDOS_∂ε = ∇_ε_LDOS(A₀, real(ω₀), vec(b))
∂LDOS_∂ω = ∇_ω_LDOS(A₀, vec(ε), real(ω₀), vec(b))

new_ω₀, _ = Eigengradient(new_ε_A, vec(ε + δε), ω, vec(b))
new_A₀, _, _ = Maxwell2d(Lx, Ly, ε + δε, real(new_ω₀); ω_pml=ω)
true_new_LDOS, _ = ∇_ε_LDOS(new_A₀, real(new_ω₀), vec(b))

true_∇LDOS = ∂LDOS_∂ε .+ ∂LDOS_∂ω .* real.(∂ω_∂ε)

@show true_new_LDOS - true_LDOS
true_∇LDOS' * vec(δε)

true_new_LDOS - true_LDOS = 3.7671334364989444e-7


3.7555465596795696e-7

In [234]:
E⁻¹ = spdiagm(1 ./ vec(ε))
C = E⁻¹ * A
u = C \ vec(b)
u_conj = conj.(u)
w = A' \ vec(b)
z = A' \ u

α = dot(u, vec(b))
β = dot(u, u)
@show ω₀ = sqrt(α / β + ω^2)
A₀, _, _ = Maxwell2d(Lx, Ly, vec(ε), real(ω₀); ω_pml=ω)
true_LDOS, ∂LDOS_∂ε = ∇_ε_LDOS(A₀, real(ω₀), vec(b))
∂LDOS_∂ω = ∇_ω_LDOS(A₀, vec(ε), real(ω₀), vec(b))

new_E⁻¹ = spdiagm(1 ./ vec(ε + δε))
new_C = new_E⁻¹ * new_ε_A
new_u = new_C \ vec(b)
new_α = dot(new_u, vec(b))
new_β = dot(new_u, new_u)
new_ω₀ = sqrt(new_α / new_β + ω^2)
new_A₀, _, _ = Maxwell2d(Lx, Ly, ε + δε, real(new_ω₀); ω_pml=ω)
true_new_LDOS, _ = ∇_ε_LDOS(new_A₀, real(new_ω₀), vec(b))

∂ω_∂ε = (1 / (2 * ω₀ * β^2)) * (β * (vec(b) + ω^2 * u_conj) .* w - 2 * α * real.((vec(b) + ω^2 * u_conj) .* z))
true_∇LDOS = ∂LDOS_∂ε .+ ∂LDOS_∂ω .* real.(∂ω_∂ε)

@show true_new_LDOS - true_LDOS
true_∇LDOS' * vec(δε)

ω₀ = sqrt(α / β + ω ^ 2) = 6.296773813612083 - 0.007041483756629559im
true_new_LDOS - true_LDOS = 3.247519933741519e-9


3.247556542032943e-9

In [ ]:
E = spdiagm(vec(ε_opt))
E⁻¹ = spdiagm(1 ./ vec(ε))
C = E⁻¹ * A
L = lu(C)

vals, vecs, _ = eigsolve(z -> L \ z, vec(b), 1, :LM, Arnoldi())
λ = 1 / vals[1]
u = vecs[1]
scale_fac = sqrt(transpose(u) * u)
u /= scale_fac
μ = (vec(b)' * u) / (λ * transpose(u) * E * u)
v = μ * u
LDOS = -imag(v' * vec(b))

new_E = spdiagm(vec(ε + δε))
new_E⁻¹ = spdiagm(1 ./ vec(ε + δε))
new_C = new_E⁻¹ * new_ε_A
new_L = lu(new_C)
new_vals, new_vecs, _ = eigsolve(z -> new_L \ z, vec(b), 1, :LM, Arnoldi())
new_λ = 1 / new_vals[1]
new_u = new_vecs[1]
new_scale_fac = sqrt(transpose(new_u) * new_u)
new_u /= new_scale_fac
new_μ = (vec(b)' * new_u) / (λ * transpose(new_u) * new_E * new_u)
new_v = new_μ * new_u
new_LDOS = -imag(new_v' * new_u)

ω₀, ∂ω_∂ε = Eigengradient(A, vec(ε), ω, vec(b))
A₀, _, _ = Maxwell2d(Lx, Ly, vec(ε), real(ω₀); ω_pml=ω)
∂LDOS_∂ε = -λ * (I - u * transpose(u)) 
∂LDOS_∂ω = ∇_ω_LDOS(A₀, vec(ε), real(ω₀), vec(b))

@show true_new_LDOS - true_LDOS
true_∇LDOS' * vec(δε)